In [1]:
import numpy as np
import pandas as pd
import os
import torch

dataset = pd.read_csv("/kaggle/input/lung-cancer/survey lung cancer.csv")
print(dataset.head())
print(dataset.describe())
print(dataset.isna().sum())
print(dataset.keys())

  GENDER  AGE  SMOKING  YELLOW_FINGERS  ANXIETY  PEER_PRESSURE  \
0      M   69        1               2        2              1   
1      M   74        2               1        1              1   
2      F   59        1               1        1              2   
3      M   63        2               2        2              1   
4      F   63        1               2        1              1   

   CHRONIC DISEASE  FATIGUE   ALLERGY   WHEEZING  ALCOHOL CONSUMING  COUGHING  \
0                1         2         1         2                  2         2   
1                2         2         2         1                  1         1   
2                1         2         1         2                  1         2   
3                1         1         1         1                  2         1   
4                1         1         1         2                  1         2   

   SHORTNESS OF BREATH  SWALLOWING DIFFICULTY  CHEST PAIN LUNG_CANCER  
0                    2                      

In [2]:
dataset = pd.concat([dataset, 
                     pd.get_dummies(dataset["GENDER"], dtype=float), 
                     pd.get_dummies(dataset["SMOKING"], dtype=float), 
                     pd.get_dummies(dataset["YELLOW_FINGERS"], dtype=float),
                     pd.get_dummies(dataset["ANXIETY"], dtype=float),
                     pd.get_dummies(dataset["PEER_PRESSURE"], dtype=float),
                     pd.get_dummies(dataset["FATIGUE "], dtype=float), 
                     pd.get_dummies(dataset["ALLERGY "], dtype=float), 
                     pd.get_dummies(dataset["WHEEZING"], dtype=float), 
                     pd.get_dummies(dataset["ALCOHOL CONSUMING"], dtype=float), 
                     pd.get_dummies(dataset["COUGHING"], dtype=float), 
                     pd.get_dummies(dataset["SHORTNESS OF BREATH"], dtype=float), 
                     pd.get_dummies(dataset["SWALLOWING DIFFICULTY"], dtype=float), 
                     pd.get_dummies(dataset["CHEST PAIN"], dtype=float)], axis=1)
dataset = dataset.drop(columns=['GENDER', 'AGE', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY',
       'PEER_PRESSURE', 'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING',
       'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH',
       'SWALLOWING DIFFICULTY', 'CHEST PAIN'])
dataset["LUNG_CANCER"] = pd.get_dummies(dataset["LUNG_CANCER"], drop_first=True, dtype=float)
dataset

,LUNG_CANCER,F,M,1,2,1,2,1,2,1,...,1,2,1,2,1,2,1,2,1,2
0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
2,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
305,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
306,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
307,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0


In [84]:
# MODEL AND TRAINING PARAMETERS
pca_coefficient = 0.95
epochs = 100
from itertools import product
grid = pd.DataFrame(list(product([1e-2, 1e-3], [1e-3, 1e-4, 1e-5], [False, True], [128, 256])), columns=['weight_decay', 'lr', 'dropout', 'hidden_size'])
grid

,weight_decay,lr,dropout,hidden_size
0,0.010,0.00100,False,128
1,0.010,0.00100,False,256
2,0.010,0.00100,True,128
3,0.010,0.00100,True,256
4,0.010,0.00010,False,128
5,0.010,0.00010,False,256
6,0.010,0.00010,True,128
7,0.010,0.00010,True,256
8,0.010,0.00001,False,128
9,0.010,0.00001,False,256


In [4]:
from torch.utils.data import Dataset
from sklearn.decomposition import PCA


class CancerDataset(Dataset):
    def __init__(self, dataset, pca_coef = .95):
        pca = PCA(pca_coef)
        dataset = dataset.rename(str,axis="columns")
        self.features = pca.fit_transform(dataset.drop(columns=["LUNG_CANCER"]))
        self.response = dataset["LUNG_CANCER"]

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = torch.FloatTensor(self.features[idx])
        response = torch.LongTensor([self.response.iloc[idx]])
        return feature, response

In [5]:
dataset = dataset.sample(frac=1)
training_data = dataset[:int(len(dataset)*0.7)]
test_data = dataset[(int(len(dataset)*0.7)+1):]

training_data = CancerDataset(training_data, pca_coefficient)
test_data = CancerDataset(test_data, pca_coefficient)

In [38]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=True)

x, y = next(iter(train_dataloader))
input_size = x.size(1)
print(x)
print(y)

tensor([[ 7.2917e-01,  2.2034e-01,  1.7644e-02,  1.6783e+00, -1.1551e+00,
         -1.2294e-01,  1.1707e-01, -1.4830e-01,  1.2047e-01,  8.9570e-01,
         -9.3556e-01],
        [-1.0265e+00,  1.8901e+00,  1.0796e+00, -7.4352e-01,  6.4291e-01,
         -3.2392e-01,  1.9213e-01,  1.0474e-01,  6.6657e-04,  1.2759e-01,
         -1.3529e-01],
        [-2.1829e+00, -8.3852e-01,  5.4567e-02, -5.5315e-01, -4.6287e-01,
         -2.5363e-01, -7.5058e-02, -3.2330e-01, -4.1366e-02, -4.2336e-01,
          6.7971e-02],
        [ 1.0200e+00, -1.2509e+00,  4.3871e-01,  4.3702e-01, -6.9413e-03,
          8.2302e-01,  8.8695e-01, -5.8980e-01, -8.5802e-01,  1.0281e-01,
         -2.1130e-01],
        [ 1.4041e+00, -7.0929e-01,  4.0902e-01, -4.5723e-01,  3.6471e-01,
         -1.0896e+00, -8.5426e-01,  5.9205e-01,  2.8393e-01, -4.8566e-01,
          1.7787e-01],
        [ 5.5129e-03,  3.2243e-01,  5.3038e-01, -7.4932e-01,  6.2062e-01,
          1.5523e+00,  1.1901e-01,  8.0067e-01, -9.8737e-01, -2.0243e-0

In [35]:
class CancerClassifier(torch.nn.Module):

    def __init__(self, input_size, hidden_size, num_classes = 2, dropout = True):
        super(CancerClassifier, self).__init__()

        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.activation = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=0.5)
        self.linear2 = torch.nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        if(self.dropout):
            x = self.dropout(x)
        x = self.linear2(x)
        return x

no_dropout_classifier = CancerClassifier(input_size = input_size, hidden_size = 128, dropout = False)

In [8]:
x, y = next(iter(train_dataloader))
torch.nn.functional.one_hot(y, num_classes = 2)[0]

tensor([[1, 0]])

In [9]:
def train(train_dataloader, optimizer, loss, model):
    losses = []
    model.train()
    
    for batch_idx, (features, response) in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        prediction = model(features)
        one_hot_response = torch.squeeze(torch.nn.functional.one_hot(response, num_classes = 2)).float()
        loss_value = loss(prediction, one_hot_response)
        losses.append(loss_value.item())
        
        loss_value.backward()
        optimizer.step()
        
    return np.mean(losses)

In [73]:
def evaluate(test_dataloader, model, loss):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for batch_idx, (features, response) in enumerate(test_dataloader):
            
            prediction = model(features)
            one_hot_response = torch.squeeze(torch.nn.functional.one_hot(response, num_classes = 2)).float()
            loss_value = loss(prediction, one_hot_response)
            test_loss += loss_value.item()
            
            correct += prediction.argmax(dim=1).eq(torch.squeeze(response)).sum().item()
            
    test_loss = test_loss / len(test_dataloader.dataset)
    accuracy = correct / len(test_dataloader.dataset)
    return test_loss, accuracy

In [85]:
data = {"loss" : [], "accuracy" : [], "dropout" : [], "weight_decay" : [], "lr" : [], "hidden_size" : []}

for i in range(len(grid)):
    model = CancerClassifier(input_size = input_size, hidden_size = grid["hidden_size"][i], dropout = grid["dropout"][i])
    
    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=grid["lr"][i], weight_decay=grid["weight_decay"][i])
    
    losses = []
    for epoch in range(epochs):
        loss_value = train(train_dataloader, optimizer, loss, model)
        losses.append(loss_value)
    test_loss, accuracy = evaluate(test_dataloader, model, loss)
    print(f"Loss: {test_loss}, and Accuracy: {accuracy}")
    
    data["accuracy"].append(accuracy)
    data["dropout"].append(grid["dropout"][i])
    data["weight_decay"].append(grid["weight_decay"][i])
    data["lr"].append(grid["lr"][i])
    data["loss"].append(test_loss)
    data["hidden_size"].append(grid["hidden_size"][i])

Loss: 0.052862410312113556, and Accuracy: 0.8043478260869565
Loss: 0.05496334867632907, and Accuracy: 0.7934782608695652
Loss: 0.0498823432818703, and Accuracy: 0.8152173913043478
Loss: 0.052563978440087776, and Accuracy: 0.8152173913043478
Loss: 0.029698982186939404, and Accuracy: 0.8586956521739131
Loss: 0.032637953110363174, and Accuracy: 0.8478260869565217
Loss: 0.03226037760791571, and Accuracy: 0.8586956521739131
Loss: 0.03092376077952592, and Accuracy: 0.8478260869565217
Loss: 0.0358090912518294, and Accuracy: 0.8586956521739131
Loss: 0.029755504558915676, and Accuracy: 0.8586956521739131
Loss: 0.035082408267518746, and Accuracy: 0.8369565217391305
Loss: 0.031909708419571754, and Accuracy: 0.8586956521739131
Loss: 0.05158539116382599, and Accuracy: 0.8043478260869565
Loss: 0.05761653411647548, and Accuracy: 0.8152173913043478
Loss: 0.05185864865779877, and Accuracy: 0.8478260869565217
Loss: 0.05633603489917258, and Accuracy: 0.8478260869565217
Loss: 0.031176180943198826, and Acc

In [86]:
df = pd.DataFrame(data = data)
df

,loss,accuracy,dropout,weight_decay,lr,hidden_size
0,0.052862,0.804348,False,0.010,0.00100,128
1,0.054963,0.793478,False,0.010,0.00100,256
2,0.049882,0.815217,True,0.010,0.00100,128
3,0.052564,0.815217,True,0.010,0.00100,256
4,0.029699,0.858696,False,0.010,0.00010,128
5,0.032638,0.847826,False,0.010,0.00010,256
6,0.032260,0.858696,True,0.010,0.00010,128
7,0.030924,0.847826,True,0.010,0.00010,256
8,0.035809,0.858696,False,0.010,0.00001,128
9,0.029756,0.858696,False,0.010,0.00001,256


In [88]:
df.to_csv("/kaggle/working/data.csv")